In [1]:
import pickle
import time
import cv2
import os
import matplotlib.pyplot as plt

import SIFT
from tqdm import tqdm

plt.set_cmap('gray')

<Figure size 432x288 with 0 Axes>

In [3]:
# load images
def load_images_from_folder(folder,color):
    images = {}
    for cdata in os.listdir(folder):
        # prevent to read something like .DS_store
        if cdata.startswith('.'):
            continue
        current_class = []
        path = folder + "/" + cdata
        for file in os.listdir(path):
            # prevent to read something like .DS_store
            if file.startswith('.'):
                continue
            image = cv2.imread(path + "/" + file)
            if color == 'gray':
                image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            else:
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            if image is not None:
                current_class.append(image)
        images[cdata] = current_class
    return images


# Read from directory
train_set  = load_images_from_folder('COMP338_Assignment1_Dataset/Training',color='gray')
test_set = load_images_from_folder('COMP338_Assignment1_Dataset/Test',color='gray')
test_set_rgb = load_images_from_folder('COMP338_Assignment1_Dataset/Test',color='rgb')

# Training process
## Step 1: Extract SIFT Feature

In [4]:
# SIFT Features
def sift_features(images):
    sift_vectors = {}
    keypoint_vectors = {}
    all_descriptor = []
    for image_class, folder in images.items():
        features = []
        keypoint_list = []
        pbar = tqdm(folder, position=0)
        for image in pbar:
            time.sleep(.01)
            pbar.set_description(f"Processing {image_class}", refresh=True)
            # Using sift in cv2 can accelerate the feature extraction powerfully
            # cv2_sift = cv2.SIFT_create(200)
            # key_p, des = cv2_sift.detectAndCompute(image, None)
            key_p, des = SIFT.computeKeypointsAndDescriptors(image)
            all_descriptor.extend(des)
            features.append(des)
            keypoint_list.append(key_p)
        sift_vectors[image_class] = features
        keypoint_vectors[image_class] = keypoint_list
    return [all_descriptor, sift_vectors, keypoint_vectors]


# Only extract first n samples
def slice_dataset(images, n):
    dic = {}
    for image_class, folder in images.items():
        folder = folder[0:n]
        dic[image_class] = folder
    return dic

In [5]:
# Extract SIFT Features from train-set
# Only use first 2 samples in developing stage
sift_train = sift_features(train_set)
# Extract SIFT Features from test-set
sift_test = sift_features(test_set)

train_des_list = sift_train[0]
train_des_by_class = sift_train[1]
train_kp_by_class = sift_train[2]

Processing keyboard: 100%|██████████| 10/10 [00:00<00:00, 49.87it/s]


### Save the SIFT feature for both training and test set

In [6]:
# transfer cv2.keypoint to dic object
def transfer_kp_list(train_kps):
    kp_list = {}
    for image_class, images in train_kps.items():
        all_image_with_kp = []
        for image in images:
            single_img = []
            for kp in image:
                if not isinstance(kp, cv2.KeyPoint):
                    return train_kps
                kp_tran = {'pt': kp.pt, 'size': kp.size}
                single_img.append(kp_tran)
            all_image_with_kp.append(single_img)
        kp_list[image_class] = all_image_with_kp
    return kp_list

train_kp_by_class = transfer_kp_list(train_kp_by_class)
test_des_by_class = sift_test[1]

In [14]:
# save numpy dict of descriptors by class
with open('Models/SIFT_feature/train_descriptors_by_class.pkl', 'wb') as f:
    pickle.dump(train_des_by_class, f)
# save dict of key_points by class
with open('Models/SIFT_feature/train_keypoint_by_class.pkl', 'wb') as f:
    pickle.dump(train_kp_by_class, f)
# save all descriptors of training set
with open('Models/SIFT_feature/all_training_descriptors.pkl', 'wb') as f:
    pickle.dump(train_des_list, f)
# save numpy dict of descriptors by class
with open('Models/SIFT_feature/test_descriptors_by_class.pkl', 'wb') as f:
    pickle.dump(test_des_by_class, f)